# Sources ([scarlet.sources](source.ipynb))

## Introduction

[scarlet.source](source.ipynb) provides classes for creating [Source](source.ipynb#scarlet.source.Source) objects,
which represent models for individual astrophysical sources.
A [Source](source.ipynb#scarlet.source.Source) is a source in a blend which is usually a single component
(a morphology and a single SED) but can also be made up of multiple components, each with it's
own SED and morphology.

### The examples below require the test data to be loaded

In [ ]:
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import scarlet
import scarlet.display

# Load the sample images
data = np.load("../data/test_sim/data.npz")
images = data["images"]
filters = data["filters"]

# Load the detection catalog
from astropy.table import Table as ApTable
catalog = ApTable.read("../data/test_sim/true_catalog.fits")
bg_rms = np.array([20]*len(images))

asinh = scarlet.display.Asinh(img=images)

## Getting Started
Typically a new source class inherits from the [Source](source.ipynb#scarlet.source.Source) base class with a specific set of constraints (see [constraints](constraints.ipynb) and an initialization function.

### ExtendedSource
[ExtendedSource](source.ipynb#scarlet.source.ExtendedSource) is a [Source](source.ipynb#scarlet.source.Source) that can be initialized to be symmetric and/or monotonoically decreasing in flux from the pixel at the center of the source.

For example:

In [ ]:
# Initialize the sources
sources = [scarlet.source.ExtendedSource((src['y'],src['x']), images, bg_rms) for src in catalog]

# Display the sources
def display_sources(sources, subset=[1,3,4]):
    # Only show a few of the peaks, for illustrative purposes
    for m in subset:
        # Load the initial source model
        src = sources[m]
        model = src.get_model()
        # Convert the model to an RGB image
        rgb = scarlet.display.img_to_rgb(model, filter_indices=[3,2,1], norm=asinh)
        # Select the image patch the overlaps with the source and convert it to an RGB image
        _rgb = scarlet.display.img_to_rgb(images[src.bb], filter_indices=[3,2,1], norm=asinh)
        # Display the image and model
        fig = plt.figure(figsize=(6,3))
        ax = [fig.add_subplot(1,2,n+1) for n in range(2)]
        ax[0].imshow(_rgb)
        ax[0].set_title("Data")
        ax[1].imshow(rgb)
        ax[1].set_title("Initial Model")
        # Mark the current source in the image
        y,x = src.center
        ax[0].plot(x-src.bb[2].start, y-src.bb[1].start, 'wx', mew=2)
        plt.show()

display_sources(sources)

By default, an [ExtendedSource](source.ipynb#scarlet.source.ExtendedSource) is constrained to be both symmetric and strictly monotonic but can be initialized with an constraint.

For example, we can initialize an extended source with no symmetry constraint but include an L0 (sparsity) penalty, however additional constraints (like sparsity) are only included during fitting, not in the initial model.

In [ ]:
import scarlet.constraints as sc
# Notice that symmetry is turned off
constraints = (
    sc.SimpleConstraint() & # Normalize SED to unity
    sc.DirectMonotonicityConstraint(use_nearest=False) & # Weighted strict monotonicity
    sc.L0Constraint(0.1) # L0 norm
)
sources = [scarlet.source.ExtendedSource((src['y'],src['x']), images, bg_rms, symmetric=False, constraints=constraints) for src in catalog]
display_sources(sources)

See [constraints](constraints.ipynb) for more on constraints

### PointSource
We can also initialize a source as a simple [PointSource](source.ipynb#scarlet.source.PointSource), which initializes each source with the SED (normalized to unity) at the location of the peak and the morphology of only a few pixels around the central (peak) pixel. Point sources also require a minimum shape to intialize the source with, although the size of the box is allowed to grow (see [scarlet.blend](blend.rst) for more on box sizing).

For example:

In [ ]:
sources = [scarlet.source.PointSource((src['y'],src['x']), images, (15,15)) for src in catalog]
display_sources(sources)

## Custom Sources
It may also be advantageous to create your own custom source initializations and constraints. For example, grand design spirals are neither monotonic nor symmetric and require less restrictive (or at the very least different) constraints.

To create your own source class you can just inherit from [Source](source.ipynb#scarlet.source.Source), for example:

In [ ]:
class MySource(scarlet.Source):
    def __init__(self, *args, **kwargs):
        # determine the initial sed and morphology, center, constraints
        # then construct the source, e.g:
        super(MySource, self).__init__(sed, morph, center=center, constraints=constraints,
                                       psf=psf, fix_sed=False, fix_morph=False, fix_frame=False, shift_center=0.2)
